In [2]:
import os
import json
import pandas as pd
import traceback

In [86]:
#from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv

In [87]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
#from langchain.callbacks import get_openai_callback
from langchain_community.callbacks import get_openai_callback
import PyPDF2
from langchain_openai import ChatOpenAI

In [5]:
load_dotenv()
key= os.getenv("CLOSEDAI_API_KEY")

In [6]:
llm= ChatOpenAI(openai_api_key=key,model_name="gpt-3.5-turbo", temperature=0.7)

/Users/mtm007/Downloads/LangChain_app/env/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [54]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [55]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject_topic} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [56]:
llm_generation_prompt = PromptTemplate(
    input_variables= ["text", "number", "subject_topic", "tone", "response_json"],
    template = TEMPLATE
)

In [57]:
llm_chain = LLMChain(llm=llm, prompt=llm_generation_prompt, output_key = "quiz", verbose=True)

In [58]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject_topic} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [59]:
quiz_evaluation_prompt = PromptTemplate(input_variables=["subject_topic","quiz"], template=TEMPLATE2)

In [60]:
review_chain = LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key = "review", verbose=True)

In [61]:
generate_evaluate_chain = SequentialChain(chains = [llm_chain, review_chain],  input_variables= ["text", "number", "subject_topic", "tone", "response_json"],
                                                    output_variables = ["quiz", "review"],verbose=True)

In [62]:
file_path = '/Users/mtm007/Downloads/LangChain_app/data.txt'
#file_path = './data.txt'

In [63]:
file_path

'/Users/mtm007/Downloads/LangChain_app/data.txt'

In [64]:
with open(file_path, 'r') as file:
    TEXT = file.read()

In [65]:
TEXT

'The dominant sequence transduction models are based on complex recurrent or\nconvolutional neural networks that include an encoder and a decoder. The best\nperforming models also connect the encoder and decoder through an attention\nmechanism. We propose a new simple network architecture, the Transformer,\nbased solely on attention mechanisms, dispensing with recurrence and convolutions\nentirely. Experiments on two machine translation tasks show these models to\nbe superior in quality while being more parallelizable and requiring significantly\nless time to train. Our model achieves 28.4 BLEU on the WMT 2014 Englishto-German translation task, improving over the existing best results, including\nensembles, by over 2 BLEU. On the WMT 2014 English-to-French translation task,\nour model establishes a new single-model state-of-the-art BLEU score of 41.0 after\ntraining for 3.5 days on eight GPUs, a small fraction of the training costs of the\nbest models from the literature.\n1 Introducti

In [66]:
#serialze python dict to json-formated string
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [67]:
NUMBER= 5,
SUBJECT_TOPIC="Deep Learning",
TONE="simple",

In [68]:
with get_openai_callback() as cb:
    response = generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject_topic": SUBJECT_TOPIC,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
    )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:The dominant sequence transduction models are based on complex recurrent or
convolutional neural networks that include an encoder and a decoder. The best
performing models also connect the encoder and decoder through an attention
mechanism. We propose a new simple network architecture, the Transformer,
based solely on attention mechanisms, dispensing with recurrence and convolutions
entirely. Experiments on two machine translation tasks show these models to
be superior in quality while being more parallelizable and requiring significantly
less time to train. Our model achieves 28.4 BLEU on the WMT 2014 Englishto-German translation task, improving over the existing best results, including
ensembles, by over 2 BLEU. On the WMT 2014 English-to-French translation task,
our model establishes a new single-model state-of-the-art BLEU score of 41.0 after
training for 3.5 days on eight GP

In [69]:
print(f"Total tokens: {cb.total_tokens}")
print(f"Prompt tokens: {cb.prompt_tokens}")
print(f"completion token: {cb.completion_tokens}")
print(f"Total cost: {cb.total_cost}")

Total tokens: 1793
Prompt tokens: 1304
completion token: 489
Total cost: 0.002934


In [70]:
quiz = response.get("quiz")

In [74]:
quiz = json.loads(quiz)

In [77]:
quiz_table_data = []
for key, value in quiz.items():
    mcq= value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option,option_value in value["options"].items()
            ]
    )
    correct = value["correct"]
    quiz_table_data.append({"MCQ":mcq, "Choices":options, "Correct":correct})

In [78]:
quiz_table_data

[{'MCQ': 'What is the Transformer network architecture based solely on?',
  'Choices': 'a: Recurrent neural networks | b: Convolutional neural networks | c: Attention mechanisms | d: Both a and b',
  'Correct': 'c'},
 {'MCQ': 'What task did the Transformer model achieve a BLEU score of 28.4 on?',
  'Choices': 'a: Image classification | b: Machine translation from English to German | c: Speech recognition | d: Text summarization',
  'Correct': 'b'},
 {'MCQ': 'How many days did it take for the Transformer model to achieve a single-model state-of-the-art BLEU score of 41.0 on the English-to-French translation task?',
  'Choices': 'a: 1 day | b: 2 days | c: 3.5 days | d: 5 days',
  'Correct': 'c'},
 {'MCQ': 'Which person proposed replacing RNNs with self-attention in the Transformer model?',
  'Choices': 'a: Jakob | b: Ashish | c: Noam | d: Niki',
  'Correct': 'a'},
 {'MCQ': 'Where was the 31st Conference on Neural Information Processing Systems (NIPS 2017) held?',
  'Choices': 'a: San Fra

In [80]:
quiz = pd.DataFrame(quiz_table_data)

In [81]:
quiz.to_csv("Transformers.csv", index=False)

In [84]:
from datetime import datetime

datetime.now().strftime('%m-%d-%Y-%H-%M-%S-')

'03-08-2024-14-27-15-'